In [1]:
%pylab --no-import-all notebook
import sys 
sys.path.append("../modules")

Populating the interactive namespace from numpy and matplotlib


In [23]:
import pandas as pd
from copy import copy
import warnings
warnings.filterwarnings('ignore')

In [3]:
import os 
path = os.getcwd()
path = path.replace(path.split('/')[-1],'')

In [4]:
valid_path = path + "Tabulky a výsledky/"

In [5]:
FS = pd.read_hdf(valid_path + '/Feature Selection/' +'FeatureSelection.h5', 'First_dataset')
FS2 = pd.read_hdf(valid_path + '/Feature Selection/' +'FeatureSelection.h5', 'Second_dataset')

In [53]:
first = pd.read_hdf(valid_path + 'Results.h5','First_dataset')
second = pd.read_hdf(valid_path + 'Results.h5','Second_dataset')

### V tomto jupyteru jsou moje poznámky k výběru příznaků z dat (Feature Selection) 

Na počátku jsem si prošel metody výběru příznaků, které jsou uvedeny na stránkách SciKit-learnu (https://scikit-learn.org/stable/modules/feature_selection.html#univariate-feature-selection)

Tyto metody jsem si vzal jako takový seznam věcí, co bych mohl vyzkoušet. 

# Jako první jsem se zaměřil na univariate ("jednorozměrné") výběry. 
Zde jsou na výběr metody jako třeba: SelectKBest, SelectPercentile atd.
Já ale tyto metody nepoužil jako takové, protože se liší jen v tom, jakým způsobem vyberou těch několik nejlepších příznaků. Proto mě spíše, než samotné metody, zajímaly způsoby, jakými jsou hodnoceny příznaky. 

Těchto způsobů je na výběr hned několik: **chi2, mutual_info_classif, f_clasif**

>**chi2** - v našem případě použít nejde, jelikož funguje jen s nezápornýmy čísly

## **mutual_info_classif**  
- resp. mutual information = vzájemná informace
- vzájemná informace mezi dvěma náhodnými veličinami nebo náhodnými vektory je míra "množství informace", získané o jedné proměnné skrze znalost (pozorování) druhé. [1]

- Vzájemnou informaci můžeme definovat, buď pomocí entropie náhodné veličiny $I(X,Y) = H(X)-H(X|Y)$, kde <br>
     \begin{equation}
     H(X) = -\sum_{x \in \mathbb{X}} p_X(x) \log p_X(x)\   \text{a}  \\
     H(X|Y) = -\sum_{y \in \mathbb{Y}} p_Y(y) \sum_{x \in \mathbb{X}} p_X(x|Y = y) \log p_X(x|Y = y). 
     \end{equation}
     nebo jako informační divergenci 
     \begin{equation}
     I(X,Y) = \sum_{y \in \mathbb{Y}}\sum_{x \in \mathbb{x}} p_{X,Y}(x,y) \log \frac{p_{X,Y}(x,y)}{p_X(x) \cdot p_Y(y)}. \text{[2]}
     \label{divergence}
     \end{equation} <br>
    
- Když X a Y jsou nezávislé, pak je vzájemná informace rovna 0. Čím větší je závislost mezi X a Y, tím je jejich vzájemná informace vyšší. 
- Vzájemná informace měří, na rozdíl od korelace, všechny _vztahy_ mezi veličinami nejenom lineární závislost. 
- Pokud za X dosadím libovolný příznak a za Y dosadím labely, pak výpočtem získáme vzájemnou informaci ("_velikost_" vztahu) mezi příznakem X a jednotlivými labely (režimy udržení) 

## **f_classif**
- jedná se o výpočet (ANOVA) F-hodnot mezi jedotlivými příznaky a labely  (stejně rozměrné vektory $X$ a $Y$ s počtem prvků $n$)<br><br>
\begin{equation}
F(X,Y) = \frac{s^2_B(X,Y)}{s^2_W(X,Y)} \ \text{, kde}
\end{equation}

\begin{equation}
s^2_W(X,Y) = \frac{1}{2n-2} \Big(  \sum_{j=1}^{n} (X_{j} - \overline{X})^2 + \sum_{j=1}^{n} (Y_{j} - \overline{Y})^2 \Big)
\end{equation}

\begin{equation}
s^2_B(X,Y) = \frac {n \cdot \big( \overline{X} - \mu \big)^2 + n \cdot \big( \overline{Y} - \mu \big)^2}{1}   
\end{equation}

\begin{equation}
\mu =\frac{\sum_{j=1}^{n} X_{j} + \sum_{j=1}^{n} Y_j}{2n}
\end{equation}

- Velká F-hodnota i-tého příznaku znamená, že tento příznak má lepší "diskriminační schopnosti". Proto je hodnotíme příznaky podle jejich F-hodnot. [3] 

- Správný postup pro výběr nejlepších příznaků:
    - určení optimálního počtu příznaků ($k$)
    - po určení počtu je třeba vytvořit $k$ skupin (podmnožin) a do nich přidáváme příznaky podle velikosti jejich F-hodnoty
        - do všech skupin dáme ten "nejlepší" příznak, pak do k-1 skupin přidáme druhý nejlepší atd. 
        
        - resp. při označení $X_1, X_2 \ \text{až} \ X_k$, kde index určuje jejich pořadí od nejlepšího k nejhoršímu. vypadají pak skupiny následovně
            - 1.skup ~ $\{X_1\}$ <br>
            - 2.skup ~ $\{X_1, X_2\}$ <br>
            - 3.skup ~ $\{X_1,X_2,X_3\}$ <br>
            - atd.
             

## FS s použitím Mutual information
- nejdříve jsem si pomocí funkce mutal_info_classif vypočítal vzájemnou informaci příznaků a labelů
- poté jsem si seřadil příznaky podle velikosti inforamce
- rozdělil jsem si příznaky do skupin "definovaných" výše
    - rozhodl jsem se použít maximálně polovinu všech příznaků tzn. $k$ = 15 
- hromadný přehled všech hodnot bude na konci sekce

### Nejdříve první dataset
- z příznaků jsem vytvořil 15 skupin, které jsem pak chtěl použít pro trénování a testování modelů (ADA,GB)
- protože chci vyzkoušet i jiné metody a tady by mě čekalo minimálně 30*10 "běhů" pro každou konfiguraci parametrů (hloubka stromů, počet stormů, ...), tak použiju jen __tři skupiny příznaků__ - sk5, sk10, sk15.  

In [7]:
mi1 = pd.read_hdf(valid_path + '/Feature Selection/' +'FeatureSelection.h5', 'First_dataset_MI')

In [8]:
mi1[[("Důležitost podle metody", "mutual_info"),
     ("Skupiny příznaků","sk 5"),
     ("Skupiny příznaků","sk 10"),
     ("Skupiny příznaků","sk 15")]].head(15)

Důležitost podle metody Skupiny příznaků            
                    mutual_info             sk 5 sk 10 sk 15
MVar 16                   0.500                1     1     1
MVar 15                   0.496                1     1     1
MVar 14                   0.487                1     1     1
MVar 13                   0.476                1     1     1
MVar 12                   0.466                1     1     1
EMM 10                    0.457                0     1     1
MVar 11                   0.454                0     1     1
EMM 12                    0.454                0     1     1
EMM 8                     0.451                0     1     1
EMM 14                    0.449                0     1     1
MM 8                      0.448                0     0     1
MM 10                     0.448                0     0     1
MM 6                      0.446                0     0     1
EMM 16                    0.445                0     0     1
MM 12                     0.444                0     0     1

#### Výsledky 

- Nejlepší výsledky každé skupiny
    - každou skupinu jsem zkoušel i z různými hyperparametry modelů a z té jsem vybral ten nejlepší výsledek pro každou skupinu
    - stejně to dělám i u ostatních tabulek níže (u MI a ANOVA)

In [9]:
pd.read_hdf(valid_path + 'Results.h5','MI_first')

Metrika       Accuracy F míra                         \
Režim                   H-mód  L-mód    ELM průměrná   
skupina model                                          
sk 5    Ada      0.829  0.866  0.748  0.803    0.806   
        GB       0.830  0.866  0.754  0.803    0.808   
sk 10   Ada      0.866  0.894  0.874  0.802    0.856   
        GB       0.869  0.895  0.887  0.805    0.862   
sk 15   Ada      0.867  0.895  0.878  0.802    0.858   
        GB       0.869  0.895  0.887  0.806    0.863   

Metrika                                             
Režim                                          std  
skupina model                                       
sk 5    Ada    [0.039, 0.045, 0.055, 0.029, 0.035]  
        GB      [0.037, 0.044, 0.048, 0.03, 0.033]  
sk 10   Ada     [0.027, 0.04, 0.036, 0.023, 0.021]  
        GB      [0.03, 0.041, 0.027, 0.037, 0.023]  
sk 15   Ada     [0.027, 0.04, 0.036, 0.024, 0.021]  
        GB     [0.031, 0.042, 0.027, 0.039, 0.025]

Na prvním datasetu bych jako nejlepší výběr označil skupinu příznaků sk10, protože má stejné skóre jako skupina sk15, ale obsahuje o 5 příznaků méně. <br>

Průměrná F-míra skupiny sk10 s použitím AdaBoost je v rámci chyby stejná jako při použití všech příznaků, ale GradientBoost má při použití všech příznaků vyšší hodnotu průměrné F-míry ( $0.893 \ \pm \ 0.024$ ). 
    - Výsledky predikce se všemi příznaky jsou v tabulkách níže


### Nyní druhý "obecnější dataset"
- opět jsem z vybral tři skupiny

In [10]:
mi2 = pd.read_hdf(valid_path + '/Feature Selection/' +'FeatureSelection.h5', 'Second_dataset_MI')

In [11]:
mi2[[("Důležitost podle metody", "mutual_info"),
     ("Skupiny příznaků","sk 5"),
     ("Skupiny příznaků","sk 10"),
     ("Skupiny příznaků","sk 15")]].head(15)

Důležitost podle metody Skupiny příznaků            
                    mutual_info             sk 5 sk 10 sk 15
MVar 16                   0.365                1     1     1
MVar 15                   0.362                1     1     1
MVar 14                   0.359                1     1     1
EMM 12                    0.359                1     1     1
MM 6                      0.358                1     1     1
MM 8                      0.357                0     1     1
EMM 10                    0.356                0     1     1
EMM 14                    0.356                0     1     1
H-alpha                   0.356                0     1     1
EMM 8                     0.355                0     1     1
MVar 13                   0.353                0     0     1
EMM 16                    0.352                0     0     1
EMM 6                     0.351                0     0     1
MM 4                      0.350                0     0     1
MM 10                     0.349                0     0     1

#### Výsledky

In [12]:
pd.read_hdf(valid_path + 'Results.h5','MI_second')

Metrika       Accuracy F míra                         \
Režim                   H-mód  L-mód    ELM průměrná   
skupina model                                          
sk 5    Ada      0.798  0.804  0.794  0.680    0.759   
        GB       0.801  0.800  0.808  0.688    0.765   
sk 10   Ada      0.804  0.809  0.803  0.689    0.767   
        GB       0.802  0.803  0.811  0.686    0.767   
sk 15   Ada      0.806  0.807  0.807  0.695    0.769   
        GB       0.800  0.801  0.810  0.685    0.765   

Metrika                                             
Režim                                          std  
skupina model                                       
sk 5    Ada    [0.122, 0.155, 0.159, 0.175, 0.114]  
        GB      [0.121, 0.157, 0.145, 0.15, 0.101]  
sk 10   Ada     [0.12, 0.152, 0.153, 0.155, 0.105]  
        GB     [0.122, 0.153, 0.143, 0.156, 0.101]  
sk 15   Ada    [0.119, 0.157, 0.148, 0.148, 0.105]  
        GB     [0.124, 0.155, 0.145, 0.156, 0.104]

Na druhém datasetu jsou opět skupiny sk10 a sk15 přibližně stejně kvalitní. Obě tyto skupiny jsou potom srovnatelné v rámci chyby s výsledkem bez výběru příznaků.
    - Výsledky predikce se všemi příznaky jsou v tabulkách níže

## FS s použitím ANOVA
- stejně jako u MI jsem si nejdříve ohodnotil jednotlivé příznaky, nyní ale pomocí f_classif
- opět jsem si seřadil příznaky podle hodnoty, rozdělil je do 15 skupin a otestoval tři z nich

### První dataset

In [13]:
f1 = pd.read_hdf(valid_path + '/Feature Selection/' + 'FeatureSelection.h5','First_dataset_ANOVA')

In [14]:
f1[[("Důležitost podle metody", "f_classif"),
    ("Skupiny příznaků","sk 5"),
    ("Skupiny příznaků","sk 10"),
    ("Skupiny příznaků","sk 15")]].head(15)

Důležitost podle metody Skupiny příznaků            
                      f_classif             sk 5 sk 10 sk 15
EMM 16                    16914                1     1     1
MM 16                     16476                1     1     1
EMM 14                    15742                1     1     1
MM 14                     15512                1     1     1
EMM 12                    14294                1     1     1
MM 12                     14194                0     1     1
EMM 10                    12646                0     1     1
MM 10                     12607                0     1     1
MVar 16                   11537                0     1     1
EMM 8                     10899                0     1     1
MM 8                      10879                0     0     1
MVar 15                   10557                0     0     1
MVar 14                    9564                0     0     1
EMM 6                      9156                0     0     1
MM 6                       9142                0     0     1

#### Výsledky

In [15]:
pd.read_hdf(valid_path + 'Results.h5','ANOVA_first')

Metrika       Accuracy F míra                         \
Režim                   H-mód  L-mód    ELM průměrná   
skupina model                                          
sk 5    Ada      0.752  0.882  0.683  0.518    0.694   
        GB       0.862  0.893  0.873  0.792    0.853   
sk 10   Ada      0.870  0.897  0.876  0.807    0.860   
        GB       0.868  0.894  0.886  0.804    0.862   
sk 15   Ada      0.870  0.897  0.879  0.808    0.861   
        GB       0.870  0.896  0.887  0.807    0.863   

Metrika                                             
Režim                                          std  
skupina model                                       
sk 5    Ada    [0.066, 0.046, 0.075, 0.084, 0.059]  
        GB     [0.032, 0.041, 0.033, 0.044, 0.026]  
sk 10   Ada     [0.025, 0.038, 0.036, 0.027, 0.02]  
        GB       [0.03, 0.04, 0.028, 0.043, 0.025]  
sk 15   Ada    [0.027, 0.039, 0.036, 0.025, 0.021]  
        GB       [0.031, 0.04, 0.029, 0.04, 0.026]

Stejně jako u vzájemné informace (MI), tak i tady u ANOVA jsou opět skupiny sk10 a sk15 stejně dobré a zase jsou srovnatelné v rámci chyby s výsledkem bez výběru příznaků.


### Druhý dataset

In [16]:
f2 = pd.read_hdf(valid_path + '/Feature Selection/' + 'FeatureSelection.h5','Second_dataset_ANOVA')

In [17]:
f2[[("Důležitost podle metody", "f_classif"),
    ("Skupiny příznaků","sk 5"),
    ("Skupiny příznaků","sk 10"),
    ("Skupiny příznaků","sk 15")]].head(15)

Důležitost podle metody Skupiny příznaků            
                      f_classif             sk 5 sk 10 sk 15
EMM 16                    10053                1     1     1
EMM 14                     9669                1     1     1
MM 16                      9621                1     1     1
MM 14                      9379                1     1     1
EMM 12                     9099                1     1     1
MM 12                      8929                0     1     1
EMM 10                     8347                0     1     1
MM 10                      8259                0     1     1
EMM 8                      7448                0     1     1
MM 8                       7406                0     1     1
EMM 6                      6446                0     0     1
MM 6                       6426                0     0     1
EMM 4                      5398                0     0     1
MM 4                       5388                0     0     1
MVar 16                    5193                0     0     1

#### Výsledky

In [18]:
pd.read_hdf(valid_path + 'Results.h5','ANOVA_second')

Metrika       Accuracy F míra                         \
Režim                   H-mód  L-mód    ELM průměrná   
skupina model                                          
sk 5    Ada      0.633  0.796  0.598  0.296    0.563   
        GB       0.783  0.790  0.793  0.630    0.738   
sk 10   Ada      0.705  0.805  0.677  0.422    0.635   
        GB       0.791  0.796  0.805  0.640    0.747   
sk 15   Ada      0.803  0.802  0.806  0.695    0.768   
        GB       0.800  0.799  0.807  0.690    0.765   

Metrika                                             
Režim                                          std  
skupina model                                       
sk 5    Ada    [0.146, 0.154, 0.142, 0.204, 0.125]  
        GB     [0.126, 0.171, 0.145, 0.207, 0.105]  
sk 10   Ada    [0.145, 0.157, 0.153, 0.241, 0.118]  
        GB     [0.122, 0.164, 0.139, 0.203, 0.101]  
sk 15   Ada     [0.121, 0.162, 0.15, 0.138, 0.107]  
        GB     [0.121, 0.155, 0.148, 0.147, 0.102]

Tato poslední tabulka jako jediná neukazuje, že skupina sk10 dosahuje stejného (resp. lepšího) skóre jako skupina sk15. 

## Vyhodnocení MI a ANOVA

- Tyto dva způsovy ohodnocení příznaků a jejich následný výběr (skupiny příznků) sice nepřispěly ke zlepšení F míry modelu, ale zjistil jsem, že při použití 10 vybraných příznaků (viz tabulky), jsem schopen získat přibližně stejnou přesnost jako při použití všech 29 příznaků a to za mnohem kratší čas. 
    - tohle platí pro oba dva modely
- Pro AdaBoost model je lepší vybírat příznaky podle vzájemné informace, protože už při 5 příznacích dodahuje F míry srovnatelné se skupinami s více příznaky.
- Oproti tomu pro GradientBoost se jako přijatelnější výběr ukázal ten podle ANOVA. Důvod je stejný jako pro MI a Adu. 

# SelectFromModel 
Po použití jednorozměrných výběrů jsem se přesunul v seznamu dále. Jako další bylo rekurentní hledání. Rekurentní hlednání, ale není nic moc zajímavého a neliší se od toho co jsem dělal dříve. Je to prostě jen vyzkoušení všech možných kombinací ( celkem $2^{29}$) s tím, že si pak vyberu tu kombinaci, se kterou měl model největší přesnost resp. F-míru.

Proto jsem rekurentní hledání přeskočil a vrhnul se rovnou na SelectFromModel. Tato metoda využívá tzv. feature_importances_ (důležitosti příznaků), které jsou vedlejším produktem některých metod. Tento atribut má třeba lineární SVC, ale taky všechny modely obsahující rozhodovaci stromy. tzv. Tree-based FS. 

Podle článku [4] se důležitost příznaku počítá podle toho, kolikrát je použit k rozdělení (splitu) a jak moc bodů rozdělí
- je normovaná na jedničku, proto se dá říct, že hodnota této důležitosti udává v _procentech_, jak moc je daný příznak používán k rozhodování
- podrobnější popis v budoucnu (potřebuju projít ten článek podrobněji)

Na základě této myšlenky můžu tedy vyhodit ty příznaky, podle kterých se málo rozhoduje a nechat si jen ty podstatné.
Zde už nepoužívám vytváření skupin příznaků jako dříve, a to z časových důvodů (trvalo by to zbytečně moc dlouho). Namísto toho používám výběr podle mediánu. 
- výběr podle mediánu je podobný předchozímu způsobu výběru, protože při použití mediánu, vyberu přesně polovinu příznaků


### Cross Fold 
Při výběru příznaků pomocí feature_importanes_ jsem narazil na malý problém. Uvědomil jsem si, že když budu vybírat příznaky s použitím celého datasetu, tak bych měl možná problém s overfitingem. Ať chci nebo ne, tak pro získání důležitosti příznků, musím natrénovat model na datech a pak ho otestovat na jiných. V tuto chvíli ale přicházím o informaci, kterou bych mohl získat z dat, na kterých testuju. Když zase použiju k tréninku i testovací data tak, by mi mohlo docházet o onomu overfitingu. 

V reálu nebudu mít k dispozici data, která budu chtít predikovat, ale při výběru příznaků, bych měl využít všeho co mám. 

Jako možné řešení problému je Cross Fold validation. Stejný princip, který používám při vyhodnocování kvality modelu si našel další uplatnění.
- rozdělím tedy data do 10 částí (já je rozdělil po signálech, každý můj dataset se skládá z 10 signálů) a střídavě určuji důležitost příznaku (při poměru 9:1). Výsledná důležitost příznaků je brána jako průměrná důležitost přes všech 10 tréninků.


## První dataset

In [19]:
FS.drop(columns=[("Důležitost podle metody", "f_classif"),
                 ("Důležitost podle metody", "mutual_info"),
                 ("Výběr podle střední hodnoty", "AdaBoost"),
                 ("Výběr podle střední hodnoty", "GB"),
                 ("Výběr podle střední hodnoty", "Random Forest")]).head()

Důležitost podle metody                      Výběr podle mediánu  \
Feature                AdaBoost     GB Random Forest            AdaBoost   
H-alpha                   0.046  0.059         0.012                True   
1.d SGF                   0.162  0.130         0.019                True   
2.d SGF                   0.066  0.100         0.007                True   
MM 4                      0.010  0.026         0.015               False   
MM 6                      0.015  0.017         0.023               False   

                              
Feature     GB Random Forest  
H-alpha   True         False  
1.d SGF   True         False  
2.d SGF   True         False  
MM 4      True         False  
MM 6     False          True

- kompletní tabulka důležitostí příznaků je na konci sekce

### Výsledky
Protože jsem se pokoušel ladit parametry ještě před pokusy o výběr příznaků, tak jsou v tabulkách níže uvedeny i tyto výsledky. Od výsledků, kde byl použit výběr se liší "hodnotou" **all**.

**all** znamená, že jsou v tomto případě použity všechy příznaky (tzn. bez výběru příznaků) 
- H-alpha: Ano
- 1.derivace Sabvitzky-Golay filtru: Ano
- 2.derivace Sabvitzky-Golay filtru: Ano
- Klouzavý průměr (MM) s délkami oken: 4,6,8,10,12,14,16
- Exponenciální klouzavý průměr (EMM) s délkami oken: 4,6,8,10,12,14,16
- Klouzavý rozptyl (MV) s délkami: 5,6,7,8,9,10,11,12,13,14,15,16

Ke zkratce **all** jsem se rozhodl, protože kdybych vypisoval všechny příznaky, tak by tabulka byla dost nepřehledná. <br>
Zde zavedená _konvence_ zkratky **all** je využívána až do konce.

#### Referenční hodnota pro první dataset
výsledky budu porovnávat s nejlepším výsledkem Skrytého Markovova Modelu

In [20]:
fd = pd.read_csv(valid_path + 'Hidden Markov Model/' + 'Cross_Fold_Modif_HMM_Hm.csv')
fd.sort_values(['F míra průměrná'], ascending=False).head(1)

,Kombinace rysů,délka úseku,Accuracy,Chyby,F míra stavu 0,F míra stavu 1,F míra stavu 2,F míra průměrná,Precision stavu 0,Precision stavu 1,Precision stavu 2,Recall stavu 0,Recall stavu 1,Recall stavu 2
4118,"(1, 1, 1, 1, 1)","(6, 10, 16, 16)",0.867752,432.2,0.896963,0.85099,0.82581,0.857921,0.922208,0.89455,0.788421,0.874542,0.828148,0.871895


##### AdaBoost
Jako první je AdaBoost. V tabulce jsou uvedeny výsledky pro dané hyperparametry (criterion, number of estimators, learning rate, max depth) a příznaky.
<br>
Tabulka je seřazena podle průměrné F míry.

In [24]:
first.loc[("Ada",)].sort_values([('F míra','průměrná')],ascending=False).head()

Metrika                                                                               Accuracy  \
Režim                                                                                            
criterion N_estim lr  MaxDepth H-alpha 1.D 2.D MM         EMM       MVar                         
default   1000    0.1 1        1       1   1   [12,14,16] [4,14,16] [5,9,13,14,15,16]    0.893   
                                               all        all       all                  0.893   
          400     0.5 1        1       1   1   [12,14,16] [4,14,16] [5,9,13,14,15,16]    0.892   
                  0.1 1        1       1   1   [12,14,16] [4,14,16] [5,9,13,14,15,16]    0.890   
                                               all        all       all                  0.891   

Metrika                                                                               F míra  \
Režim                                                                                  H-mód   
criterion N_estim lr  MaxDepth H-alpha 1.D 2.D MM         EMM       MVar                       
default   1000    0.1 1        1       1   1   [12,14,16] [4,14,16] [5,9,13,14,15,16]  0.915   
                                               all        all       all                0.915   
          400     0.5 1        1       1   1   [12,14,16] [4,14,16] [5,9,13,14,15,16]  0.914   
                  0.1 1        1       1   1   [12,14,16] [4,14,16] [5,9,13,14,15,16]  0.914   
                                               all        all       all                0.914   

Metrika                                                                                       \
Režim                                                                                  L-mód   
criterion N_estim lr  MaxDepth H-alpha 1.D 2.D MM         EMM       MVar                       
default   1000    0.1 1        1       1   1   [12,14,16] [4,14,16] [5,9,13,14,15,16]  0.894   
                                               all        all       all                0.894   
          400     0.5 1        1       1   1   [12,14,16] [4,14,16] [5,9,13,14,15,16]  0.891   
                  0.1 1        1       1   1   [12,14,16] [4,14,16] [5,9,13,14,15,16]  0.895   
                                               all        all       all                0.894   

Metrika                                                                                       \
Režim                                                                                    ELM   
criterion N_estim lr  MaxDepth H-alpha 1.D 2.D MM         EMM       MVar                       
default   1000    0.1 1        1       1   1   [12,14,16] [4,14,16] [5,9,13,14,15,16]  0.848   
                                               all        all       all                0.848   
          400     0.5 1        1       1   1   [12,14,16] [4,14,16] [5,9,13,14,15,16]  0.848   
                  0.1 1        1       1   1   [12,14,16] [4,14,16] [5,9,13,14,15,16]  0.839   
                                               all        all       all                0.840   

Metrika                                                                                         \
Režim                                                                                 průměrná   
criterion N_estim lr  MaxDepth H-alpha 1.D 2.D MM         EMM       MVar                         
default   1000    0.1 1        1       1   1   [12,14,16] [4,14,16] [5,9,13,14,15,16]    0.886   
                                               all        all       all                  0.886   
          400     0.5 1        1       1   1   [12,14,16] [4,14,16] [5,9,13,14,15,16]    0.884   
                  0.1 1        1       1   1   [12,14,16] [4,14,16] [5,9,13,14,15,16]    0.883   
                                               all        all       all                  0.883   

Metrika                                                                                                                    

##### Gradient boosting

In [25]:
first.loc[("GB",)].sort_values([('F míra','průměrná')],ascending=False).head()

Metrika                                                                                         Accuracy  \
Režim                                                                                                      
criterion N_estim lr  MaxDepth H-alpha 1.D 2.D MM          EMM             MVar                            
default   400     0.1 3        1       1   1   [4,16]      [4,16]          [5,6,11,13,14,15,16]    0.900   
                                               all         all             all                     0.899   
          1000    0.1 3        1       1   1   [4,16]      [4,16]          [5,6,11,13,14,15,16]    0.898   
                                               all         all             all                     0.899   
          400     0.1 3        0       0   0   [6,8,10,12] [8,10,12,14,16] [11,12,13,14,15,16]     0.869   

Metrika                                                                                         F míra  \
Režim                                                                                            H-mód   
criterion N_estim lr  MaxDepth H-alpha 1.D 2.D MM          EMM             MVar                          
default   400     0.1 3        1       1   1   [4,16]      [4,16]          [5,6,11,13,14,15,16]  0.922   
                                               all         all             all                   0.921   
          1000    0.1 3        1       1   1   [4,16]      [4,16]          [5,6,11,13,14,15,16]  0.920   
                                               all         all             all                   0.921   
          400     0.1 3        0       0   0   [6,8,10,12] [8,10,12,14,16] [11,12,13,14,15,16]   0.895   

Metrika                                                                                                 \
Režim                                                                                            L-mód   
criterion N_estim lr  MaxDepth H-alpha 1.D 2.D MM          EMM             MVar                          
default   400     0.1 3        1       1   1   [4,16]      [4,16]          [5,6,11,13,14,15,16]  0.895   
                                               all         all             all                   0.895   
          1000    0.1 3        1       1   1   [4,16]      [4,16]          [5,6,11,13,14,15,16]  0.894   
                                               all         all             all                   0.892   
          400     0.1 3        0       0   0   [6,8,10,12] [8,10,12,14,16] [11,12,13,14,15,16]   0.887   

Metrika                                                                                                 \
Režim                                                                                              ELM   
criterion N_estim lr  MaxDepth H-alpha 1.D 2.D MM          EMM             MVar                          
default   400     0.1 3        1       1   1   [4,16]      [4,16]          [5,6,11,13,14,15,16]  0.862   
                                               all         all             all                   0.860   
          1000    0.1 3        1       1   1   [4,16]      [4,16]          [5,6,11,13,14,15,16]  0.859   
                                               all         all             all                   0.860   
          400     0.1 3        0       0   0   [6,8,10,12] [8,10,12,14,16] [11,12,13,14,15,16]   0.806   

Metrika                                                                                                   \
Režim                                                                                           průměrná   
criterion N_estim lr  MaxDepth H-alpha 1.D 2.D MM          EMM             MVar                            
default   400     0.1 3        1       1   1   [4,16]      [4,16]          [5,6,11,13,14,15,16]    0.893   
                                               all         all             all                     0.892   
          1000    0.1 3        1       1   1   [

##### Random Forests
- jelikož pro RF jsem používal větší množství možných hyperparametrů, tak v tabulce níže je pouze 5 nejlepších konfigurací
    - tyto konfigurace jsou po výběru příznaků. 
- u RF se jako u jediného modelu ukázal výběr příznaků, zbytečný, neboť nejlepších výsledků dosahují se všemi příznaky viz druhá tabulka níže

In [26]:
first.loc[(first.index.get_level_values('MM')!="all")\
          &(first.index.get_level_values('Model')=="RF")].sort_values([('F míra','průměrná')],ascending=False).head()

Metrika                                                                                                    Accuracy  \
Režim                                                                                                                 
Model criterion N_estim lr   MaxDepth H-alpha 1.D 2.D MM             EMM               MVar                           
RF    entropy   90      None 5        0       0   0   [6,8,10,12,14] [6,8,10,12,14,16] [11,12,13,14,15,16]     0.87   
                50      None 5        0       0   0   [6,8,10,12,14] [6,8,10,12,14,16] [11,12,13,14,15,16]     0.87   
                220     None 5        0       0   0   [6,8,10,12,14] [6,8,10,12,14,16] [11,12,13,14,15,16]     0.87   
                210     None 5        0       0   0   [6,8,10,12,14] [6,8,10,12,14,16] [11,12,13,14,15,16]     0.87   
                10      None 5        0       0   0   [6,8,10,12,14] [6,8,10,12,14,16] [11,12,13,14,15,16]     0.87   

Metrika                                                                                                    F míra  \
Režim                                                                                                       H-mód   
Model criterion N_estim lr   MaxDepth H-alpha 1.D 2.D MM             EMM               MVar                         
RF    entropy   90      None 5        0       0   0   [6,8,10,12,14] [6,8,10,12,14,16] [11,12,13,14,15,16]  0.898   
                50      None 5        0       0   0   [6,8,10,12,14] [6,8,10,12,14,16] [11,12,13,14,15,16]  0.898   
                220     None 5        0       0   0   [6,8,10,12,14] [6,8,10,12,14,16] [11,12,13,14,15,16]  0.899   
                210     None 5        0       0   0   [6,8,10,12,14] [6,8,10,12,14,16] [11,12,13,14,15,16]  0.898   
                10      None 5        0       0   0   [6,8,10,12,14] [6,8,10,12,14,16] [11,12,13,14,15,16]  0.898   

Metrika                                                                                                            \
Režim                                                                                                       L-mód   
Model criterion N_estim lr   MaxDepth H-alpha 1.D 2.D MM             EMM               MVar                         
RF    entropy   90      None 5        0       0   0   [6,8,10,12,14] [6,8,10,12,14,16] [11,12,13,14,15,16]  0.885   
                50      None 5        0       0   0   [6,8,10,12,14] [6,8,10,12,14,16] [11,12,13,14,15,16]  0.886   
                220     None 5        0       0   0   [6,8,10,12,14] [6,8,10,12,14,16] [11,12,13,14,15,16]  0.886   
                210     None 5        0       0   0   [6,8,10,12,14] [6,8,10,12,14,16] [11,12,13,14,15,16]  0.886   
                10      None 5        0       0   0   [6,8,10,12,14] [6,8,10,12,14,16] [11,12,13,14,15,16]  0.885   

Metrika                                                                                                            \
Režim                                                                                                         ELM   
Model criterion N_estim lr   MaxDepth H-alpha 1.D 2.D MM             EMM               MVar                         
RF    entropy   90      None 5        0       0   0   [6,8,10,12,14] [6,8,10,12,14,16] [11,12,13,14,15,16]  0.805   
                50      None 5        0       0   0   [6,8,10,12,14] [6,8,10,12,14,16] [11,12,13,14,15,16]  0.805   
                220     None 5        0       0   0   [6,8,10,12,14] [6,8,10,12,14,16] [11,12,13,14,15,16]  0.805   
                210     None 5        0       0   0   [6,8,10,12,14] [6,8,10,12,14,16] [11,12,13,14,15,16]  0.805   
                10      None 5        0       0   0   [6,8,10,12,14] [6,8,10,12,14,16] [11,12,13,14,15,16]  0.805   

Metrika                                                                                                              \
Režim                                                                                                      průměrná  

In [27]:
first.loc[("RF",)].sort_values([('F míra','průměrná')],ascending=False).head()

Metrika                                                      Accuracy F míra  \
Režim                                                                  H-mód   
criterion N_estim lr   MaxDepth H-alpha 1.D 2.D MM  EMM MVar                   
entropy   10      None 5        1       1   1   all all all     0.882  0.909   
          30      None 5        1       1   1   all all all     0.882  0.910   
          105     None 5        1       1   1   all all all     0.880  0.908   
          45      None 5        1       1   1   all all all     0.880  0.908   
          40      None 5        1       1   1   all all all     0.880  0.908   

Metrika                                                                     \
Režim                                                         L-mód    ELM   
criterion N_estim lr   MaxDepth H-alpha 1.D 2.D MM  EMM MVar                 
entropy   10      None 5        1       1   1   all all all   0.889  0.826   
          30      None 5        1       1   1   all all all   0.888  0.825   
          105     None 5        1       1   1   all all all   0.889  0.821   
          45      None 5        1       1   1   all all all   0.888  0.822   
          40      None 5        1       1   1   all all all   0.889  0.821   

Metrika                                                                \
Režim                                                        průměrná   
criterion N_estim lr   MaxDepth H-alpha 1.D 2.D MM  EMM MVar            
entropy   10      None 5        1       1   1   all all all     0.875   
          30      None 5        1       1   1   all all all     0.874   
          105     None 5        1       1   1   all all all     0.873   
          45      None 5        1       1   1   all all all     0.873   
          40      None 5        1       1   1   all all all     0.873   

Metrika                                                                                            
Režim                                                                                         std  
criterion N_estim lr   MaxDepth H-alpha 1.D 2.D MM  EMM MVar                                       
entropy   10      None 5        1       1   1   all all all   [0.028, 0.037, 0.024, 0.029, 0.022]  
          30      None 5        1       1   1   all all all   [0.029, 0.037, 0.024, 0.029, 0.022]  
          105     None 5        1       1   1   all all all    [0.027, 0.035, 0.023, 0.025, 0.02]  
          45      None 5        1       1   1   all all all    [0.03, 0.038, 0.023, 0.029, 0.022]  
          40      None 5        1       1   1   all all all   [0.029, 0.037, 0.023, 0.027, 0.022]

### Vyhodnocení
- všechny nově použité metody (RF, AdaBoost, GradientBoosting) překonaly v průměrné F-míře původní HMM
    - i nejhorší RF je v rámci chyby stejně kvalitní jako HMM (nechci říkat přesný protože neporovnávám podle přesnosti ale podle F-míry)
- jako nejlepší z nic se ukázal GB s průměrnou F-mírou $0.893 \pm 0.024$
- u všech metod se s použitím vybraných příznaků nezměnila kvalita modelu (stejné hodnoty u většiny metrik v rámci chyby), ale zkrátil se čas potřebný k predikci.
    - do času predikce počítám čas potřebný k výpočtu všech potřebných příznaků + čas predikce jako takové 

## Druhý dataset

In [28]:
FS2.drop(columns=[("Důležitost podle metody", "f_classif"),
                  ("Důležitost podle metody", "mutual_info"),
                  ("Výběr podle střední hodnoty", "AdaBoost"),
                  ("Výběr podle střední hodnoty", "GB"),
                  ("Výběr podle střední hodnoty", "Random Forest")]).head()

Důležitost podle metody                      Výběr podle mediánu  \
Feature                AdaBoost     GB Random Forest            AdaBoost   
H-alpha                   0.052  0.056         0.009                True   
1.d SGF                   0.161  0.135         0.015                True   
2.d SGF                   0.074  0.102         0.005                True   
MM 4                      0.035  0.022         0.016                True   
MM 6                      0.030  0.025         0.021                True   

                             
Feature    GB Random Forest  
H-alpha  True         False  
1.d SGF  True         False  
2.d SGF  True         False  
MM 4     True         False  
MM 6     True         False

#### Referenční hodnota pro druhý dataset

In [29]:
sd = pd.read_csv(valid_path + 'Hidden Markov Model/' + 'Cross_Fold_Modif_HMM_Obecne.csv')
sd.sort_values(['F míra průměrná'], ascending=False).head(1)

,Kombinace rysů,délka úseku,Accuracy,Chyby,F míra stavu 0,F míra stavu 1,F míra stavu 2,F míra průměrná,Precision stavu 0,Precision stavu 1,Precision stavu 2,Recall stavu 0,Recall stavu 1,Recall stavu 2
2206,"(1, 0, 0, 1, 1)","(4, 8, 16, 16)",0.81153,606.3,0.793359,0.763951,0.683584,0.746965,0.857688,0.797517,0.639613,0.748688,0.778536,0.770808


#### AdaBoost

In [30]:
second.loc[("Ada",)].sort_values([('F míra','průměrná')],ascending=False).head()

Metrika                                                                             Accuracy  \
Režim                                                                                          
criterion N_estim lr  MaxDepth H-alpha 1.D 2.D MM               EMM    MVar                    
default   1000    0.1 1        1       1   1   [4,6,8,12,14,16] [4,16] [7,10,15,16]    0.827   
                                               all              all    all             0.827   
          400     0.5 1        1       1   1   [4,6,8,12,14,16] [4,16] [7,10,15,16]    0.821   
                  0.1 1        1       1   1   [4,6,8,12,14,16] [4,16] [7,10,15,16]    0.824   
                                               all              all    all             0.826   

Metrika                                                                             F míra  \
Režim                                                                                H-mód   
criterion N_estim lr  MaxDepth H-alpha 1.D 2.D MM               EMM    MVar                  
default   1000    0.1 1        1       1   1   [4,6,8,12,14,16] [4,16] [7,10,15,16]  0.826   
                                               all              all    all           0.826   
          400     0.5 1        1       1   1   [4,6,8,12,14,16] [4,16] [7,10,15,16]  0.823   
                  0.1 1        1       1   1   [4,6,8,12,14,16] [4,16] [7,10,15,16]  0.824   
                                               all              all    all           0.825   

Metrika                                                                                     \
Režim                                                                                L-mód   
criterion N_estim lr  MaxDepth H-alpha 1.D 2.D MM               EMM    MVar                  
default   1000    0.1 1        1       1   1   [4,6,8,12,14,16] [4,16] [7,10,15,16]  0.820   
                                               all              all    all           0.818   
          400     0.5 1        1       1   1   [4,6,8,12,14,16] [4,16] [7,10,15,16]  0.814   
                  0.1 1        1       1   1   [4,6,8,12,14,16] [4,16] [7,10,15,16]  0.819   
                                               all              all    all           0.818   

Metrika                                                                                     \
Režim                                                                                  ELM   
criterion N_estim lr  MaxDepth H-alpha 1.D 2.D MM               EMM    MVar                  
default   1000    0.1 1        1       1   1   [4,6,8,12,14,16] [4,16] [7,10,15,16]  0.716   
                                               all              all    all           0.715   
          400     0.5 1        1       1   1   [4,6,8,12,14,16] [4,16] [7,10,15,16]  0.714   
                  0.1 1        1       1   1   [4,6,8,12,14,16] [4,16] [7,10,15,16]  0.705   
                                               all              all    all           0.707   

Metrika                                                                                       \
Režim                                                                               průměrná   
criterion N_estim lr  MaxDepth H-alpha 1.D 2.D MM               EMM    MVar                    
default   1000    0.1 1        1       1   1   [4,6,8,12,14,16] [4,16] [7,10,15,16]    0.787   
                                               all              all    all             0.786   
          400     0.5 1        1       1   1   [4,6,8,12,14,16] [4,16] [7,10,15,16]    0.784   
                  0.1 1        1       1   1   [4,6,8,12,14,16] [4,16] [7,10,15,16]    0.783   
                                               all              all    all             0.783   

Metrika                                                                                                                   
Režim                                                                           

#### Gradient Boosting

In [31]:
second.loc[("GB",)].sort_values([('F míra','průměrná')],ascending=False).head()

Metrika                                                                              Accuracy  \
Režim                                                                                           
criterion N_estim lr  MaxDepth H-alpha 1.D 2.D MM     EMM          MVar                         
default   400     0.1 3        1       1   1   [4,16] [4,6,16]     [5,6,11,14,15,16]    0.833   
                                               all    all          all                  0.832   
          1000    0.1 3        1       1   1   [4,16] [4,6,16]     [5,6,11,14,15,16]    0.831   
                                               all    all          all                  0.829   
          400     0.1 3        1       0   0   [6,8]  [8,10,12,14] [14,15,16]           0.802   

Metrika                                                                              F míra  \
Režim                                                                                 H-mód   
criterion N_estim lr  MaxDepth H-alpha 1.D 2.D MM     EMM          MVar                       
default   400     0.1 3        1       1   1   [4,16] [4,6,16]     [5,6,11,14,15,16]  0.837   
                                               all    all          all                0.837   
          1000    0.1 3        1       1   1   [4,16] [4,6,16]     [5,6,11,14,15,16]  0.835   
                                               all    all          all                0.834   
          400     0.1 3        1       0   0   [6,8]  [8,10,12,14] [14,15,16]         0.803   

Metrika                                                                                      \
Režim                                                                                 L-mód   
criterion N_estim lr  MaxDepth H-alpha 1.D 2.D MM     EMM          MVar                       
default   400     0.1 3        1       1   1   [4,16] [4,6,16]     [5,6,11,14,15,16]  0.816   
                                               all    all          all                0.817   
          1000    0.1 3        1       1   1   [4,16] [4,6,16]     [5,6,11,14,15,16]  0.816   
                                               all    all          all                0.813   
          400     0.1 3        1       0   0   [6,8]  [8,10,12,14] [14,15,16]         0.811   

Metrika                                                                                      \
Režim                                                                                   ELM   
criterion N_estim lr  MaxDepth H-alpha 1.D 2.D MM     EMM          MVar                       
default   400     0.1 3        1       1   1   [4,16] [4,6,16]     [5,6,11,14,15,16]  0.725   
                                               all    all          all                0.722   
          1000    0.1 3        1       1   1   [4,16] [4,6,16]     [5,6,11,14,15,16]  0.720   
                                               all    all          all                0.715   
          400     0.1 3        1       0   0   [6,8]  [8,10,12,14] [14,15,16]         0.686   

Metrika                                                                                        \
Režim                                                                                průměrná   
criterion N_estim lr  MaxDepth H-alpha 1.D 2.D MM     EMM          MVar                         
default   400     0.1 3        1       1   1   [4,16] [4,6,16]     [5,6,11,14,15,16]    0.793   
                                               all    all          all                  0.792   
          1000    0.1 3        1       1   1   [4,16] [4,6,16]     [5,6,11,14,15,16]    0.790   
                                               all    all          all                  0.788   
          400     0.1 3        1       0   0   [6,8]  [8,10,12,14] [14,15,16]           0.767   

Metrika                                                                                                                    
Režim                                  

#### Random Forest
- i u druhého datasetu se "prokázalo", že RF dosahuje lepších výsledků se všemi přiznaky 

In [32]:
second.loc[(second.index.get_level_values('MM')!="all")\
          &(second.index.get_level_values('Model')=="RF")].sort_values([('F míra','průměrná')],ascending=False).head()

Metrika                                                                                                    Accuracy  \
Režim                                                                                                                 
Model criterion N_estim lr   MaxDepth H-alpha 1.D 2.D MM             EMM               MVar                           
RF    entropy   240     None 3        0       0   0   [8,10]         [6,8,10,12,14,16] [11,12,13,14,15,16]    0.804   
                200     None 3        0       0   0   [8,10]         [6,8,10,12,14,16] [11,12,13,14,15,16]    0.803   
                             5        0       0   0   [6,8,10,12,14] [6,8,10,12,14,16] [11,12,13,14,15,16]    0.803   
                120     None 3        0       0   0   [8,10]         [6,8,10,12,14,16] [11,12,13,14,15,16]    0.803   
                             5        0       0   0   [6,8,10,12,14] [6,8,10,12,14,16] [11,12,13,14,15,16]    0.803   

Metrika                                                                                                    F míra  \
Režim                                                                                                       H-mód   
Model criterion N_estim lr   MaxDepth H-alpha 1.D 2.D MM             EMM               MVar                         
RF    entropy   240     None 3        0       0   0   [8,10]         [6,8,10,12,14,16] [11,12,13,14,15,16]  0.810   
                200     None 3        0       0   0   [8,10]         [6,8,10,12,14,16] [11,12,13,14,15,16]  0.810   
                             5        0       0   0   [6,8,10,12,14] [6,8,10,12,14,16] [11,12,13,14,15,16]  0.810   
                120     None 3        0       0   0   [8,10]         [6,8,10,12,14,16] [11,12,13,14,15,16]  0.811   
                             5        0       0   0   [6,8,10,12,14] [6,8,10,12,14,16] [11,12,13,14,15,16]  0.811   

Metrika                                                                                                            \
Režim                                                                                                       L-mód   
Model criterion N_estim lr   MaxDepth H-alpha 1.D 2.D MM             EMM               MVar                         
RF    entropy   240     None 3        0       0   0   [8,10]         [6,8,10,12,14,16] [11,12,13,14,15,16]  0.799   
                200     None 3        0       0   0   [8,10]         [6,8,10,12,14,16] [11,12,13,14,15,16]  0.799   
                             5        0       0   0   [6,8,10,12,14] [6,8,10,12,14,16] [11,12,13,14,15,16]  0.799   
                120     None 3        0       0   0   [8,10]         [6,8,10,12,14,16] [11,12,13,14,15,16]  0.798   
                             5        0       0   0   [6,8,10,12,14] [6,8,10,12,14,16] [11,12,13,14,15,16]  0.798   

Metrika                                                                                                            \
Režim                                                                                                         ELM   
Model criterion N_estim lr   MaxDepth H-alpha 1.D 2.D MM             EMM               MVar                         
RF    entropy   240     None 3        0       0   0   [8,10]         [6,8,10,12,14,16] [11,12,13,14,15,16]  0.673   
                200     None 3        0       0   0   [8,10]         [6,8,10,12,14,16] [11,12,13,14,15,16]  0.675   
                             5        0       0   0   [6,8,10,12,14] [6,8,10,12,14,16] [11,12,13,14,15,16]  0.675   
                120     None 3        0       0   0   [8,10]         [6,8,10,12,14,16] [11,12,13,14,15,16]  0.672   
                             5        0       0   0   [6,8,10,12,14] [6,8,10,12,14,16] [11,12,13,14,15,16]  0.672   

Metrika                                                                                                              \
Režim                                                                                                      průměrná  

In [33]:
second.loc[("RF",)].sort_values([('F míra','průměrná')],ascending=False).head()

Metrika                                                      Accuracy F míra  \
Režim                                                                  H-mód   
criterion N_estim lr   MaxDepth H-alpha 1.D 2.D MM  EMM MVar                   
entropy   15      None 5        1       1   1   all all all     0.818  0.827   
          40      None 5        1       1   1   all all all     0.817  0.826   
          110     None 5        1       1   1   all all all     0.817  0.826   
          45      None 5        1       1   1   all all all     0.816  0.826   
          55      None 5        1       1   1   all all all     0.816  0.826   

Metrika                                                                     \
Režim                                                         L-mód    ELM   
criterion N_estim lr   MaxDepth H-alpha 1.D 2.D MM  EMM MVar                 
entropy   15      None 5        1       1   1   all all all   0.807  0.694   
          40      None 5        1       1   1   all all all   0.808  0.690   
          110     None 5        1       1   1   all all all   0.808  0.692   
          45      None 5        1       1   1   all all all   0.807  0.691   
          55      None 5        1       1   1   all all all   0.806  0.689   

Metrika                                                                \
Režim                                                        průměrná   
criterion N_estim lr   MaxDepth H-alpha 1.D 2.D MM  EMM MVar            
entropy   15      None 5        1       1   1   all all all     0.776   
          40      None 5        1       1   1   all all all     0.775   
          110     None 5        1       1   1   all all all     0.775   
          45      None 5        1       1   1   all all all     0.774   
          55      None 5        1       1   1   all all all     0.774   

Metrika                                                                                            
Režim                                                                                         std  
criterion N_estim lr   MaxDepth H-alpha 1.D 2.D MM  EMM MVar                                       
entropy   15      None 5        1       1   1   all all all   [0.113, 0.142, 0.135, 0.175, 0.098]  
          40      None 5        1       1   1   all all all   [0.112, 0.142, 0.135, 0.176, 0.097]  
          110     None 5        1       1   1   all all all   [0.113, 0.143, 0.135, 0.172, 0.098]  
          45      None 5        1       1   1   all all all    [0.114, 0.142, 0.135, 0.17, 0.098]  
          55      None 5        1       1   1   all all all   [0.112, 0.142, 0.135, 0.173, 0.097]

**all** znamená, že jsou v tomto případě použity všechy příznaky (tzn. bez výběru příznaků) 
- H-alpha: Ano
- 1.derivace Sabvitzky-Golay filtru: Ano
- 2.derivace Sabvitzky-Golay filtru: Ano
- Klouzavý průměr (MM) s délkami oken: 4,6,8,10,12,14,16
- Exponenciální klouzavý průměr (EMM) s délkami oken: 4,6,8,10,12,14,16
- Klouzavý rozptyl (MV) s délkami: 5,6,7,8,9,10,11,12,13,14,15,16

### Vyhodnocení
- Stejně jako u prvního datasetu všechny metody překonaly původní HMM. 
- Výběrem příznaků pomocí důležitosti jsem snížil počet příznaků na polovinu, ale F-míra u Ady a GB je totožná se hodnotou dosaženou se všemi pří.
- jako nejlepší z nich se ukázal GB s průměrnou F-mírou $0.793 \pm 0.108$


## Kompletní tabulka hodnocení příznaků pro všechny metody

### První dataset

In [34]:
FS.drop(columns=[("Výběr podle střední hodnoty", "AdaBoost"),
                 ("Výběr podle střední hodnoty", "GB"),
                 ("Výběr podle střední hodnoty", "Random Forest"),
                 ("Výběr podle mediánu", "AdaBoost"),
                 ("Výběr podle mediánu", "GB"),\
                 ("Výběr podle mediánu", "Random Forest")])

Důležitost podle metody                                          
Feature               f_classif mutual_info AdaBoost     GB Random Forest
H-alpha                    5298       0.418    0.046  0.059         0.012
1.d SGF                      74       0.315    0.162  0.130         0.019
2.d SGF                     250       0.228    0.066  0.100         0.007
MM 4                       7465       0.437    0.010  0.026         0.015
MM 6                       9142       0.446    0.015  0.017         0.023
MM 8                      10879       0.448    0.011  0.017         0.036
MM 10                     12607       0.448    0.005  0.016         0.040
MM 12                     14194       0.444    0.024  0.018         0.037
MM 14                     15512       0.432    0.059  0.017         0.022
MM 16                     16476       0.418    0.083  0.051         0.015
EMM 4                      7474       0.426    0.032  0.028         0.014
EMM 6                      9156       0.441    0.014  0.024         0.023
EMM 8                     10899       0.451    0.014  0.012         0.047
EMM 10                    12646       0.457    0.003  0.008         0.063
EMM 12                    14294       0.454    0.003  0.011         0.081
EMM 14                    15742       0.449    0.036  0.019         0.091
EMM 16                    16914       0.445    0.050  0.034         0.103
MVar 5                     2276       0.307    0.019  0.035         0.003
MVar 6                     2903       0.339    0.007  0.023         0.004
MVar 7                     3575       0.368    0.017  0.020         0.005
MVar 8                     4297       0.396    0.008  0.016         0.007
MVar 9                     5069       0.418    0.018  0.018         0.011
MVar 10                    5894       0.437    0.012  0.017         0.017
MVar 11                    6763       0.454    0.010  0.023         0.023
MVar 12                    7659       0.466    0.002  0.022         0.032
MVar 13                    8594       0.476    0.019  0.027         0.043
MVar 14                    9564       0.487    0.018  0.031         0.055
MVar 15                   10557       0.496    0.021  0.023         0.071
MVar 16                   11537       0.500    0.212  0.156         0.083

### Druhý dataset

In [35]:
FS2.drop(columns=[("Výběr podle střední hodnoty", "AdaBoost"),
                 ("Výběr podle střední hodnoty", "GB"),
                 ("Výběr podle střední hodnoty", "Random Forest"),
                 ("Výběr podle mediánu", "AdaBoost"),
                 ("Výběr podle mediánu", "GB"),\
                 ("Výběr podle mediánu", "Random Forest")])

Důležitost podle metody                                          
Feature               f_classif mutual_info AdaBoost     GB Random Forest
H-alpha                    3906       0.356    0.052  0.056         0.009
1.d SGF                      73       0.225    0.161  0.135         0.015
2.d SGF                     247       0.163    0.074  0.102         0.005
MM 4                       5388       0.350    0.035  0.022         0.016
MM 6                       6426       0.358    0.030  0.025         0.021
MM 8                       7406       0.357    0.029  0.020         0.036
MM 10                      8259       0.349    0.009  0.017         0.032
MM 12                      8929       0.341    0.030  0.016         0.024
MM 14                      9379       0.339    0.041  0.019         0.014
MM 16                      9621       0.328    0.075  0.059         0.013
EMM 4                      5398       0.346    0.017  0.039         0.018
EMM 6                      6446       0.351    0.009  0.022         0.032
EMM 8                      7448       0.355    0.008  0.023         0.049
EMM 10                     8347       0.356    0.007  0.014         0.068
EMM 12                     9099       0.359    0.003  0.010         0.082
EMM 14                     9669       0.356    0.007  0.014         0.088
EMM 16                    10053       0.352    0.061  0.038         0.071
MVar 5                     1352       0.223    0.015  0.035         0.004
MVar 6                     1705       0.255    0.010  0.017         0.006
MVar 7                     2069       0.276    0.017  0.020         0.008
MVar 8                     2444       0.294    0.009  0.017         0.012
MVar 9                     2827       0.311    0.009  0.019         0.016
MVar 10                    3215       0.334    0.017  0.016         0.024
MVar 11                    3597       0.338    0.012  0.024         0.034
MVar 12                    3959       0.344    0.010  0.019         0.044
MVar 13                    4305       0.353    0.010  0.020         0.050
MVar 14                    4633       0.359    0.010  0.024         0.063
MVar 15                    4930       0.362    0.019  0.024         0.071
MVar 16                    5193       0.365    0.215  0.136         0.076

# Nakonec kompletní tabulka výsledků všech konfigurací 
srovnaná podle průměrné F-míry

## První dataset

In [56]:
first.sort_values([('F míra','průměrná')],ascending=False)

Metrika                                                                                                        Accuracy  \
Režim                                                                                                                     
Model criterion N_estim lr   MaxDepth H-alpha 1.D 2.D MM                EMM               MVar                            
GB    default   400     0.1  3        1       1   1   [4,16]            [4,16]            [5,6,11,13,14,15,16]    0.900   
                                                      all               all               all                     0.899   
                1000    0.1  3        1       1   1   all               all               all                     0.899   
                                                      [4,16]            [4,16]            [5,6,11,13,14,15,16]    0.898   
Ada   default   1000    0.1  1        1       1   1   [12,14,16]        [4,14,16]         [5,9,13,14,15,16]       0.893   
                                                      all               all               all                     0.893   
                400     0.5  1        1       1   1   [12,14,16]        [4,14,16]         [5,9,13,14,15,16]       0.892   
                        0.1  1        1       1   1   [12,14,16]        [4,14,16]         [5,9,13,14,15,16]       0.890   
                                                      all               all               all                     0.891   
                        0.5  1        1       1   1   all               all               all                     0.888   
                        1.0  1        1       1   1   [12,14,16]        [4,14,16]         [5,9,13,14,15,16]       0.886   
RF    entropy   10      None 5        1       1   1   all               all               all                     0.882   
                30      None 5        1       1   1   all               all               all                     0.882   
                60      None 5        1       1   1   all               all               all                     0.881   
                25      None 5        1       1   1   all               all               all                     0.881   
                35      None 5        1       1   1   all               all               all                     0.880   
                45      None 5        1       1   1   all               all               all                     0.880   
                105     None 5        1       1   1   all               all               all                     0.880   
                40      None 5        1       1   1   all               all               all                     0.880   
                20      None 5        1       1   1   all               all               all                     0.881   
                135     None 5        1       1   1   all               all               all                     0.879   
                15      None 5        1       1   1   all               all               all                     0.879   
                185     None 5        1       1   1   all               all               all                     0.880   
                205     None 5        1       1   1   all               all               all                     0.880   
                175     None 5        1       1   1   all               all               all                     0.880   
                165     None 5        1       1   1   all               all               all                     0.880   
                215     None 5        1       1   1   all               all               all                     0.880   
                150     None 5        1       1   1   all               all               all                     0.880   
                100     None 5        1       1   1   all               all               all                     0.880   
                125     None 5        1       1   1   all       

## Druhý dataset

In [57]:
second.sort_values([('F míra','průměrná')],ascending=False)

Metrika                                                                                                      Accuracy  \
Režim                                                                                                                   
Model criterion N_estim lr   MaxDepth H-alpha 1.D 2.D MM               EMM               MVar                           
GB    default   400     0.1  3        1       1   1   [4,16]           [4,6,16]          [5,6,11,14,15,16]      0.833   
                                                      all              all               all                    0.832   
                1000    0.1  3        1       1   1   [4,16]           [4,6,16]          [5,6,11,14,15,16]      0.831   
                                                      all              all               all                    0.829   
Ada   default   1000    0.1  1        1       1   1   [4,6,8,12,14,16] [4,16]            [7,10,15,16]           0.827   
                                                      all              all               all                    0.827   
                400     0.5  1        1       1   1   [4,6,8,12,14,16] [4,16]            [7,10,15,16]           0.821   
                        0.1  1        1       1   1   all              all               all                    0.826   
                                                      [4,6,8,12,14,16] [4,16]            [7,10,15,16]           0.824   
                        0.5  1        1       1   1   all              all               all                    0.821   
RF    entropy   15      None 5        1       1   1   all              all               all                    0.818   
                40      None 5        1       1   1   all              all               all                    0.817   
                110     None 5        1       1   1   all              all               all                    0.817   
                225     None 5        1       1   1   all              all               all                    0.816   
                220     None 5        1       1   1   all              all               all                    0.816   
                215     None 5        1       1   1   all              all               all                    0.816   
                45      None 5        1       1   1   all              all               all                    0.816   
                125     None 5        1       1   1   all              all               all                    0.817   
                230     None 5        1       1   1   all              all               all                    0.817   
                55      None 5        1       1   1   all              all               all                    0.816   
                120     None 5        1       1   1   all              all               all                    0.816   
                35      None 5        1       1   1   all              all               all                    0.816   
                115     None 5        1       1   1   all              all               all                    0.816   
                180     None 5        1       1   1   all              all               all                    0.816   
                25      None 5        1       1   1   all              all               all                    0.816   
                145     None 5        1       1   1   all              all               all                    0.816   
                100     None 5        1       1   1   all              all               all                    0.816   
                75      None 5        1       1   1   all              all               all                    0.816   
                200     None 5        1       1   1   all              all               all                    0.816   
                90      None 5        1       1   1   all              all               all                    0.817   
...    

# Refs:
[1] C. KRIER, D. FRANCOIS, etc.: _Feature scoring by mutual information for classification of mass spectra_ <br>
[2] I. Vajda: _Teorie Informace_<br>
[3] H. Ding, etc.: _Identification of bacteriophage virion proteins with the ANOVA feature selection and analysis_ <br>
[4] G. Louppe, etc.: _Understanding variable importances in forests of randomized trees_<br>